# Predicción con Series de Tiempo

En este laboratorio se implementan 5 métodos diferentes para predecir con series de tiempo, utilizando 4 data sets distintos.

In [1]:
# se cargan las librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
# se cargan los data sets
df_1 = pd.read_csv("daily-total-female-births.csv")
df_2 = pd.read_csv("monthly-car-sales.csv")
df_3 = pd.read_csv("monthly-mean-temp.csv")
df_4 = pd.read_csv("shampoo.csv")

In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    365 non-null    object
 1   Births  365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [4]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Month   108 non-null    object
 1   Sales   108 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [5]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Month        240 non-null    object 
 1   Temperature  240 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.9+ KB


In [6]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Month   36 non-null     object 
 1   Sales   36 non-null     float64
dtypes: float64(1), object(1)
memory usage: 704.0+ bytes


#### PREPARACIÓN

In [7]:
def division_entreno_prueba(datos, n_prueba):
    return datos[: -n_prueba], datos[-n_prueba:]

def medir_rmse(actual, predicho):
    return sqrt(mean_squared_error(actual, predicho))

def validacion_al_frente(datos, n_prueba):
    predicciones = [ ]
    entreno, prueba = division_entreno_prueba(datos, n_prueba)
    modelo = model_fit(entreno)
    historia = [x for x in entreno]
    # Caminar hacia adelante
    for i in range(len(test)):
        # Ajustar el modelo y predecir
        yhat = model_predict(modelo, historia)
        predicciones.append(yhat)
        historia.append(prueba[i])
    # Estimar el error
    error = medir_rmse(prueba, predicciones)
    print("> {error:.3f}")
    return historia

#### 1. Promedio

In [9]:
def prediccion_promedio(historia, config):
    n, tipo_prom = config
    if tipo_prom == "prom":
        return mean(historia[-n:])
    return median(historia[-n:])

#### 2. SARIMAX

In [10]:
def prediccion_sarima(historia, orden, orden_estacional, tendencia):
    modelo = sm.tsa.statespace.SARIMAX(historia, order = orden, seasona_order = orden_estacional,
                     trend = tendencia, enforce_stationarity = False, enforce_invertibility = False)
    modelo_ajustado = modelo.fit(disp = False)
    yhat = modelo_ajustado.predict(len(historia), len(historia))
    return yhat[0]

In [11]:
historia_1 = validacion_al_frente(df_1, 182)
sarimax_1 = prediccion_sarima(historia_1,(7,1,7),(1,0,5,1),'n')

NameError: name 'model_fit' is not defined

#### 3. Winter-Holt

In [12]:
def AlisamientoExponencial(historia, t, d, s, p, b, r):
    historia = array(historia)
    modelo = ExponentialSmoothing(historia, trend = t, damped = d,
                                  seasonal = s, seasonal_periods = p)
    modelo_ajustado = modelo.fit(optimized = True, use_boxcox = b, remove_bias = r)
    yhat = modelo_ajustado.predict(len(historia), len(historia))
    return yhat[0]

#### 4. Red Neuronal

In [13]:
#Transformación de series
def series_a_supervisado(datos, n_entrada, n_salida = 1):
    df = DataFrame(datos)
    columnas = [ ]
    for i in range(n_entrada, 0, -1):
        columnas.append(df.shift(-1))
    for i in range(0, n_salida):
        columnas.appemd(df.shift(-1))
    agregado = concat(columnas, axis = 1)
    agregado.dropna(inplace = true)
    return agregado.values

#Predicción con Tensorflow y Keras
def prediccion_red_neuronal(historia, n_entrada, n_nodos, n_epocas, tamanio_tandas):
    datos = series_a_supervisado(historia, n_entrada)
    entreno_X, entreno_y = datos[:, :-1], datos[:, -1]
    modelo = Sequential()
    modelo.add(Dense(n_nodes, activation='relu', input_dim=n_entrada))
    modelo.add(layers.Dense(1))
    modelo.compile(loss = "mse", optimizer = "adam")
    modelo_ajustado = modelo.fit(entreno_X, entreno_y, epochs = n_epocas,
    batch_size = tamanio_tandas, verbose = 0)
    x_entrada = array(historia[-n_entrada:]).reshape(1, n_entrada)
    yhat = modelo.predict(x_entrada, verbose = 0)
    return yhat[0]

#### 5. FB Prophet

In [19]:
from prophet import Prophet

ModuleNotFoundError: No module named 'prophet'

In [14]:
#Requiere de una pequeña transformación de los datos a usar
series = pd.read_csv("monthly-car-sales.csv", header = 0, index_col = None)
series["ds"] = pd.to_datetime(series["Month"])
series["y"] = series[["Sales"]].astype(float)
series = series[["ds", "y"]]
series.head( )

#Ejecución
modelo = Prophet(mcmc_samples = 500, seasonality_mode = "multiplicative")
modelo_ajustado = modelo.fit(series)
futuro = modelo_ajustado.make_future_dataframe(periods = 48, freq = "M")
prediccion = modelo_ajustado.predict(futuro)
prediccion[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail( )

NameError: name 'Prophet' is not defined